# 3.2 Incompressible Navier-Stokes equations

As in [the whetting the appetite demo](../wta/navierstokes.ipynb) we consider the unsteady Navier Stokes equations

Find $(u,p):[0,T] \to (H_{0,D}^1)^d \times L^2$, s.t.

\begin{align*}
\int_{\Omega} \partial_t u \cdot v + \int_{\Omega} \nu \nabla u \nabla v + u \cdot \nabla u v - \int_{\Omega} \operatorname{div}(v) p &= \int f v  && \forall v \in (H_{0,D}^1)^d, \\ 
- \int_{\Omega} \operatorname{div}(u) q &= 0 && \forall q \in L^2, \\
\quad u(t=0) & = u_0
\end{align*}

### Schäfer-Turek benchmark
We consider the benchmark setup of from http://www.featflow.de/en/benchmarks/cfdbenchmarking/flow/dfg_benchmark2_re100.html . The geometry is a 2D channel with a circular obstacle which is positioned (only slightly) off the center of the channel. The geometry:

![title](geometry.png) 

The viscosity is set to $\nu = 10^{-3}$.

In [1]:
# import libarries, define geometry and generate mesh
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.webgui import Draw as DrawGeo
from math import pi

maxh  = 0.05
mesh = Mesh(unit_square.GenerateMesh(maxh=maxh))
h = specialcf.mesh_size
n = specialcf.normal(2)

In [2]:
nu = 3e-6
#dt = 0.1
dt = 0.025*maxh
gammau = 0.001

In [3]:
# dirichlet="left|right|bottom|top"

In [4]:
tstart = 0.0 
t = Parameter(tstart)
u1 =  CoefficientFunction(t*t)
u_t = u1.Diff(t)
u_t(mesh(0.0))

0.0

In [5]:
u_exact = CoefficientFunction((cos(t)*(2*sin(pi*x)**2*y*(1-y)*(1-2*y)),
                          cos(t)*(-pi*sin(2*pi*x)*y**2*(1-y)**2 ) ) )
p_exact = CoefficientFunction(cos(t)*sin(pi*x)*cos(pi*y) )


In [1]:
div_sol = u_exact[0].Diff(x) + u_exact[1].Diff(y) 
#div_sol(mesh(0.4,0.4))
div_square_L2 = Integrate(div_sol**2,mesh,order=12)
print("div_square_L2 = ", div_square_L2)

NameError: name 'u_exact' is not defined

In [7]:
rhs = CoefficientFunction(( u_exact[0].Diff(t) + (u_exact[0]*u_exact[0].Diff(x) + u_exact[1]*u_exact[0].Diff(y)) 
                           - nu* ( u_exact[0].Diff(x).Diff(x) + u_exact[0].Diff(y).Diff(y) ) +  p_exact.Diff(x) ,
                           u_exact[1].Diff(t) + (u_exact[0]*u_exact[1].Diff(x) + u_exact[1]*u_exact[1].Diff(y)) 
                           - nu* ( u_exact[1].Diff(x).Diff(x) + u_exact[1].Diff(y).Diff(y) ) +  p_exact.Diff(y)
                          ) )

In [8]:
mean_pressure = Integrate(p_exact,mesh,order=12)
print("mean pressure = ", mean_pressure) 

mean pressure =  -8.565197162635485e-17


In [9]:
#DrawGeo (shape)
Draw (mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Viscosity:

### Taylor-Hood velocity-pressure pair

We use a Taylor-Hood discretization of degree $k$. The finite element space for the (vectorial) velocity $\mathbf{u}$ and the pressure $p$ are:

\begin{align}
\mathbf{u} \in \mathbf{V} &= \{ v \in H^1(\Omega) | v|_T \in \mathcal{P}^k(T) \}^2 \\
p \in Q &= \{ q \in H^1(\Omega) | q|_T \in \mathcal{P}^{k-1}(T) \}
\end{align}

In [10]:
k = 2
V1 = H1(mesh,order=1, dirichlet="left|right|bottom|top") 
V = VectorH1(mesh,order=k, dirichlet="left|right|bottom|top", dgjumps=True)
Q = H1(mesh,order=k)
L = NumberSpace(mesh)
#X = V*Q*L
Qavg = Q*L 

The `VectorH1` is the product space of two `H1` spaces with convenience operators for identity, `grad`(`Grad`) and `div`. 

### Stokes problem for initial values
Find $\mathbf{u} \in \mathbf{V}$, $p \in Q$ so that

\begin{align}
\int_{\Omega} \nu \nabla \mathbf{u} : \nabla \mathbf{v} - \int_{\Omega} \operatorname{div}(\mathbf{v}) p &= \int \mathbf{f}  \cdot \mathbf{v}  && \forall \mathbf{v} \in \mathbf{V}, \\- \int_{\Omega} \operatorname{div}(\mathbf{u}) q &= 0 && \forall q \in Q,
\end{align}

with boundary conditions:

 * the inflow boundary data ('inlet') with mean value $1$
 
\begin{align}
u_x(0,y) &= 6 y(0.41-y)/(0.41)^2, \quad \int_{0}^{0.41} u_x(0,y) dy = 1 \\
u_y(0,y) &= 0
\end{align}
 
 * "do-nothing" boundary conditions on the ouflow ('outlet') and
 * homogeneous Dirichlet conditions on all other boundaries ('wall').

In [11]:
gfu = GridFunction(V)
u, v = V.TnT() 
(p,lam), (q,mu) = Qavg.TnT() 

jump_u = grad(u) -  grad(u).Other()
jump_v =  grad(v) -  grad(v).Other()

u_0 = GridFunction(V)
u_1 = GridFunction(V)
ustar = GridFunction(V)
dustar = GridFunction(V)
custar = GridFunction(V)
pavg = GridFunction(Qavg)

# Set initial values using L2-projection
t.Set(0.0*dt)

mmass = BilinearForm(V)
mmass += InnerProduct(u,v)*dx
mmass.Assemble()
invm = mmass.mat.Inverse(V.FreeDofs())

aq = BilinearForm(Qavg)
aq += (InnerProduct( grad(p) , grad(q) ) + mu * p + lam * q)  * dx
aq.Assemble()
invaq = aq.mat.Inverse(Qavg.FreeDofs())

am = BilinearForm(V)
am += (  (1/dt)*InnerProduct(u,v) + 0.5* nu * InnerProduct(grad(u),grad(v)) ) *dx
am.Assemble()
inva = am.mat.Inverse(V.FreeDofs())

S_1 = BilinearForm(V)
S_1 += gammau * h**2 * (  IfPos(u*n, u*n, -u*n) ) * InnerProduct (jump_u, jump_v  ) * dx(skeleton=True)
#S_1 += gammau * h**2  * InnerProduct(u,n) * InnerProduct (jump_u, jump_v  ) * dx(skeleton=True)
S_1.Assemble() 

S_2 = BilinearForm(V)
S_2 += gammau * h**2 * InnerProduct (jump_u, jump_v  ) * dx(skeleton=True)
S_2.Assemble()

anu = BilinearForm(V)
anu += nu*InnerProduct(grad(u),grad(v)) * dx
anu.Assemble()

for i,ui in zip(range(2),[u_0,u_1]):
    t.Set(i*dt)
    fm = LinearForm(V)   
    fm += InnerProduct( u_exact , v) * dx
    fm.Assemble()
    ui.vec.data = invm * fm.vec

#Draw(u_0 ,mesh,"u0")
#Draw(u_1 ,mesh,"u1")



velocity = gfu
#velocity, pressure, meanp = gfu.components
# parabolic inflow at bc=1:
#uin = CoefficientFunction((1.5*4*y*(0.41-y)/(0.41*0.41),0))
#velocity.Set(uin, definedon=mesh.Boundaries("inlet"))

#Draw(velocity,mesh,"u")
#Draw(pressure,mesh,"p")

#Draw(u_exact,mesh,"u-exact")
#Draw( p_exact,mesh,"p-exact")

In [12]:
beta_norm = GridFunction(V1)
beta_norm.Set(sqrt(InnerProduct(u_0,u_0 )) )
print( max(beta_norm.vec.FV().NumPy()[:]))
Draw(beta_norm ,mesh,"beta")

0.19750185272549042


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

#### inflow profile plot:

To solve the Stokes (and later the Navier-Stokes) problem, we introduce the bilinear form:

$$
  a((u,p),(v,q)) := \int_{\Omega} \nu \nabla u : \nabla v - \operatorname{div}(v) p - \operatorname{div}(u) q
$$

and directly use it to solve the Stokes problem for suitable initial values:

In [13]:
# (mu * p + lam * q)

In [14]:


#a = BilinearForm(X)
#stokes = (nu*InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p + mu * p + lam * q)*dx
#a += stokes
#a.Assemble()

#f = LinearForm(V)   
#f += InnerProduct(rhs , v) * dx
#f.Assemble()

#res = f.vec - a.mat*gfu.vec
#inv_stokes = a.mat.Inverse(X.FreeDofs())
#gfu.vec.data += inv_stokes * res

In [15]:
#sceneu = Draw(velocity,mesh,"u")
#scenep = Draw(pressure,mesh,"p")

## IMEX time discretization
For the time integration we consider an semi-implicit Euler method (IMEX) where the convection is treated only explicitly and the remaining part implicitly:

Find $(\mathbf{u}^{n+1},p^{n+1}) \in X = \mathbf{V} \times Q$, s.t. for all $(\mathbf{v},q) \in X = \mathbf{V} \times Q$

\begin{align}
\underbrace{m(\mathbf{u}^{n+1},\mathbf{v}) + \Delta t ~\cdot~a((\mathbf{u}^{n+1},p^{n+1}),(\mathbf{v},q))}_{ \to M^\ast} ~=~m(\mathbf{u}^{n},\mathbf{v}) - \Delta t ~\cdot~c(\mathbf{u}^{n}; \mathbf{u}^{n},\mathbf{v}) 
\end{align}

with 
\begin{align}
m(\mathbf{u},\mathbf{v}) = \int \mathbf{u} \cdot \mathbf{v}
\end{align}

and 
\begin{align}
c(\mathbf{w},\mathbf{u},\mathbf{v}) = \int \mathbf{w} \cdot \nabla \mathbf{u} \cdot \mathbf{v}
\end{align}

We prefer the incremental form (as it homogenizes the boundary conditions):
\begin{align}
m(\delta \mathbf{u}^{n+1},\mathbf{v}) + \Delta t ~\cdot~a((\delta \mathbf{u}^{n+1},\delta p^{n+1}),(\mathbf{v},q)) ~=~ \Delta t (-a((\mathbf{u}^{n},p^n),(\mathbf{v},q)) -c(\mathbf{u}^{n}; \mathbf{u}^{n},\mathbf{v}))
\end{align}

In [16]:

# matrix for implicit part of IMEX(1) scheme:
#mstar = BilinearForm(V)
#mstar += InnerProduct(u,v)*dx + dt*stokes
#mstar.Assemble()
#inv = mstar.mat.Inverse(X.FreeDofs())



Next, we want to evaluate the convection term $c(u^n;u^n,v)$ for given $u^n$, i.e. a linear form. We can do this by setting up a `LinearForm`
```
fc = LinearForm(X)
fc += (Grad(gfu) * gfu) * v * dx
```
Then, for $u^n =$`gfu` a call `fc.Assemble()` yields a vector `fc.vec`representing the linear form. 

Alternatively, we can use the semi-linear form structure of the convection form and put it into a `BilinearForm` (which indeed is able to represent non-bilinear forms as we will discuss in more detail in [unit-3.3](../unit-3.3-nonlinear/nonlinear.ipynb)). We will not need the assembly (or the storage of a sparse matrix) though:

In [17]:
conv = BilinearForm(V, nonassemble = True)
conv += (Grad(u) * u) * v * dx
#conv.Assemble()

Instead of assembling a `LinearForm` in every time step, we use an operator application of a `BilinearForm`, see [unit-2.11](../unit-2.11-matrixfree/matrixfree.ipynb) or the next units for more details.

The `BilinearForm`(which indeed is rather a semi-linear form) can compute the linear form at a given state `u`:

 * either by a call `conv.Apply(gfu.vec,res)` where the result ends up in the vector `res`
 * alternatively we can use the notation of an operator application `conv.mat * gfu.vec` yielding the same.

Below, we will use the latter.

In [18]:
f_d = LinearForm(V)
f_d += ( (-1) * InnerProduct(rhs , v) ) * dx

f_nod = LinearForm(V) 
f_nod += ( (-1) * InnerProduct(rhs , v) ) * dx

R_C = LinearForm(V)
R_C.Assemble()
tmp = R_C.vec.CreateVector()

In [19]:
tend = 0
gfut = GridFunction(gfu.space,multidim=0)

In [20]:
# implicit Euler/explicit Euler splitting method:
tend += 1
gfut.AddMultiDimComponent(gfu.vec)
tref = dt; cnt = 1
#t.Set(dt+0.5*dt)
while t < tend-0.5*dt:
    print ("\rt=", t, end="")
    # Step 1
    ustar.vec.data = 1.5*u_1.vec 
    ustar.vec.data -= 0.5*u_0.vec
    #conv.Assemble()
    
    #beta_norm.Set(sqrt(InnerProduct(ustar.components[0],ustar.components[0] )) )
    beta_norm.Set(sqrt(InnerProduct(ustar,ustar )) )
    beta_infty = max(beta_norm.vec.FV().NumPy()[:])
    print("beta_infty = ", beta_infty )
    
    #jump_ustar = grad(ustar) -  grad(ustar).Other()
    #jump_v =  grad(v) -  grad(v).Other()
    
    t.Set(tref+0.5*dt)
    f_d.Assemble()
    
    #f_d += (  nu*InnerProduct(grad(ustar),grad(v)) ) * dx
    #f_d += gammau * h**2 * (  IfPos(ustar*n, ustar*n, -ustar*n) + 0.05*beta_infty  ) * InnerProduct (jump_ustar, jump_v  ) * dx(skeleton=True)
    #f_d += gammau * h**2 * InnerProduct (jump_ustar, jump_v  ) * dx(skeleton=True)
        
    res =  f_d.vec
    res += anu.mat * ustar.vec
    res += conv.mat * ustar.vec
    res += S_1.mat * ustar.vec
    res +=  beta_infty * 0.05 * S_2.mat * ustar.vec
    dustar.vec.data =  invm * res

    # Preparatory for Step 4   
    #f_nod = LinearForm(V) 
    #f_nod += ( (-1) * InnerProduct(rhs , v) ) * dx
    #f_nod += gammau * h**2 * (  sqrt((ustar*n)**2) + 0.05*beta_infty  ) * InnerProduct (jump_ustar, jump_v  ) * dx(skeleton=True)
    f_nod.Assemble() 
    f_nod.vec.data += conv.mat * ustar.vec
    f_nod.vec.data += S_1.mat * ustar.vec
    f_nod.vec.data += beta_infty * 0.05 * S_2.mat * ustar.vec
    custar.vec.data =  invm * f_nod.vec
    
    # Step 2 
    R_D = LinearForm(Qavg)
    R_D += ( InnerProduct(-dustar,grad(q)) + (1/dt)* InnerProduct( u_1 ,grad(q) ) ) *dx
    R_D.Assemble()
    
    # Step 3 
    pavg.vec.data = invaq * R_D.vec
    p1 = pavg.components[0]
    
    # Step 4 
    #R_C = LinearForm(V)
    R_C += ( InnerProduct(-custar,v)  + (1/dt)* InnerProduct( u_1 , v ) ) *dx
    #R_C.Assemble()
    tmp.data = -custar.vec + (1/dt)*u_1.vec
    R_C.vec.data =  mmass.mat * tmp
    #R_C.vec.data += (1/dt) * 
    
    
    f_4 = LinearForm(V)  
    f_4 +=  - nu*0.5*InnerProduct( grad( u_1 ) , grad(v)) * dx 
    f_4 += - InnerProduct( grad(p1) , v) * dx 
    f_4.Assemble()
    f_4.vec.data += R_C.vec 
    
    u_0.vec.data = u_1.vec 
    u_1.vec.data = inva * f_4.vec
    
    tref = tref + dt; cnt += 1
    
    
    #f = LinearForm(X)   
    #f += InnerProduct(rhs , v) * dx
    #f += InnerProduct(rhs , v) * dx
    #res += mmass.mat * gfu.vec
    #res -= - dt * conv.mat * gfu.vec
    #gfu.vec.data = inv * res    
    
    t.Set(tref) 
    l2_error_vel = sqrt(Integrate( InnerProduct( u_1 - u_exact , u_1 - u_exact  ) ,mesh ))
    t.Set(tref-0.5*dt) 
    l2_error_pressure = sqrt(Integrate( InnerProduct( p1 - p_exact , p1 - p_exact  ) ,mesh ))
    print("L2-error=velocity = ",l2_error_vel )
    print("L2-error=pressure = ",l2_error_pressure )
    
    #sceneu.Redraw()
    #scenep.Redraw()
    #if cnt % 50 == 0:
    #    gfut.AddMultiDimComponent(gfu.vec)

t= ParameterCF, val = 0.00125
beta_infty =  0.19750162127803741
L2-error=velocity =  0.00012372385295365316
L2-error=pressure =  0.004284408176207938
t= ParameterCF, val = 0.001875
beta_infty =  0.19750135784787437
L2-error=velocity =  0.00017626032002760557
L2-error=pressure =  0.0010227238581375852
t= ParameterCF, val = 0.003125
beta_infty =  0.19749925657235592
L2-error=velocity =  0.0002111572473733292
L2-error=pressure =  0.0008459826343249651
t= ParameterCF, val = 0.004375
beta_infty =  0.19749787946045222
L2-error=velocity =  0.00023785828661030276
L2-error=pressure =  0.0007873916541630906
t= ParameterCF, val = 0.005625
beta_infty =  0.19749620237026677
L2-error=velocity =  0.0002598780640681513
L2-error=pressure =  0.0007538615644801747
t= ParameterCF, val = 0.006875
beta_infty =  0.19749427425868366
L2-error=velocity =  0.0002788826977101271
L2-error=pressure =  0.0007307600227298082
t= ParameterCF, val = 0.008125
beta_infty =  0.19749207181542286
L2-error=velocity =  0.00029

L2-error=velocity =  0.0006494520873534526
L2-error=pressure =  0.0005326379357204638
t= ParameterCF, val = 0.069375
beta_infty =  0.1970239513184623
L2-error=velocity =  0.0006538687922506985
L2-error=pressure =  0.0005313102192984665
t= ParameterCF, val = 0.070625
beta_infty =  0.19700708171723713
L2-error=velocity =  0.0006582260096699824
L2-error=pressure =  0.000530012483735728
t= ParameterCF, val = 0.071875
beta_infty =  0.19698991174267283
L2-error=velocity =  0.000662525005037447
L2-error=pressure =  0.0005287436931190768
t= ParameterCF, val = 0.073125
beta_infty =  0.1969724412034599
L2-error=velocity =  0.0006667669964876226
L2-error=pressure =  0.0005275028616234694
t= ParameterCF, val = 0.074375
beta_infty =  0.19695466992991098
L2-error=velocity =  0.0006709531577419123
L2-error=pressure =  0.0005262890502485033
t= ParameterCF, val = 0.075625
beta_infty =  0.1969365977722644
L2-error=velocity =  0.0006750846207432655
L2-error=pressure =  0.0005251013638192667
t= ParameterC

L2-error=velocity =  0.0008270326308537745
L2-error=pressure =  0.0004872676238284701
t= ParameterCF, val = 0.138125
beta_infty =  0.1956486237916755
L2-error=velocity =  0.0008293465028151576
L2-error=pressure =  0.00048675946839287843
t= ParameterCF, val = 0.139375
beta_infty =  0.1956151791879809
L2-error=velocity =  0.0008316361036925699
L2-error=pressure =  0.0004862579200588601
t= ParameterCF, val = 0.140625
beta_infty =  0.19558143338040845
L2-error=velocity =  0.000833901754572367
L2-error=pressure =  0.00048576282470553606
t= ParameterCF, val = 0.141875
beta_infty =  0.1955473863756617
L2-error=velocity =  0.0008361437708513727
L2-error=pressure =  0.00048527403211287286
t= ParameterCF, val = 0.143125
beta_infty =  0.1955130381804661
L2-error=velocity =  0.0008383624623784209
L2-error=pressure =  0.0004847913958389809
t= ParameterCF, val = 0.144375
beta_infty =  0.19547838880161875
L2-error=velocity =  0.0008405581335909105
L2-error=pressure =  0.0004843147731015873
t= Paramet

L2-error=velocity =  0.0009264605474075949
L2-error=pressure =  0.00046579559183850124
t= ParameterCF, val = 0.208125
beta_infty =  0.19331224177808493
L2-error=velocity =  0.0009277901794557122
L2-error=pressure =  0.0004654963510647869
t= ParameterCF, val = 0.209375
beta_infty =  0.19326195677501373
L2-error=velocity =  0.0009291074365882555
L2-error=pressure =  0.00046519887850869965
t= ParameterCF, val = 0.210625
beta_infty =  0.1932113720937936
L2-error=velocity =  0.0009304124534165654
L2-error=pressure =  0.0004649031306549976
t= ParameterCF, val = 0.211875
beta_infty =  0.19316048779514405
L2-error=velocity =  0.0009317053626748047
L2-error=pressure =  0.0004646090648665462
t= ParameterCF, val = 0.213125
beta_infty =  0.19310930394106335
L2-error=velocity =  0.0009329862952498809
L2-error=pressure =  0.00046431663936397517
t= ParameterCF, val = 0.214375
beta_infty =  0.19305782059481885
L2-error=velocity =  0.000934255380210727
L2-error=pressure =  0.00046402581320616974
t= Par

L2-error=velocity =  0.0009850021194981002
L2-error=pressure =  0.0004508146662365376
t= ParameterCF, val = 0.278125
beta_infty =  0.19003687403593764
L2-error=velocity =  0.0009858031092093056
L2-error=pressure =  0.0004505659862342259
t= ParameterCF, val = 0.279375
beta_infty =  0.18996993032181098
L2-error=velocity =  0.0009865970256572952
L2-error=pressure =  0.00045031757938254427
t= ParameterCF, val = 0.280625
beta_infty =  0.1899026918394683
L2-error=velocity =  0.000987383932003898
L2-error=pressure =  0.0004500694307400301
t= ParameterCF, val = 0.281875
beta_infty =  0.1898351586933973
L2-error=velocity =  0.0009881638905869542
L2-error=pressure =  0.0004498215256334621
t= ParameterCF, val = 0.283125
beta_infty =  0.18976733098823476
L2-error=velocity =  0.0009889369629316848
L2-error=pressure =  0.00044957384965152095
t= ParameterCF, val = 0.284375
beta_infty =  0.18969920882874372
L2-error=velocity =  0.0009897032097618919
L2-error=pressure =  0.0004493263886410159
t= Parame

...

L2-error=velocity =  0.0010379255807085582
L2-error=pressure =  0.0003491854539572377
t= ParameterCF, val = 0.698125
beta_infty =  0.1515207535851259
L2-error=velocity =  0.001037684571437311
L2-error=pressure =  0.0003488021992392111
t= ParameterCF, val = 0.699375
beta_infty =  0.15136039722386063
L2-error=velocity =  0.0010374419375135092
L2-error=pressure =  0.00034841846878468893
t= ParameterCF, val = 0.700625
beta_infty =  0.15119979285142587
L2-error=velocity =  0.001037197683090148
L2-error=pressure =  0.00034803426311857293
t= ParameterCF, val = 0.701875
beta_infty =  0.15103894079255598
L2-error=velocity =  0.0010369518123107095
L2-error=pressure =  0.0003476495827683114
t= ParameterCF, val = 0.703125
beta_infty =  0.15087784137308446
L2-error=velocity =  0.001036704329309226
L2-error=pressure =  0.00034726442826390965
t= ParameterCF, val = 0.704375
beta_infty =  0.1507164949199298
L2-error=velocity =  0.0010364552382104006
L2-error=pressure =  0.00034687880013787556
t= Parame

beta_infty =  0.14234170805579882
L2-error=velocity =  0.0010220390261596968
L2-error=pressure =  0.0003270060100481102
t= ParameterCF, val = 0.768125
beta_infty =  0.14216821789621104
L2-error=velocity =  0.0010217131969978979
L2-error=pressure =  0.00032659699017828823
t= ParameterCF, val = 0.769375
beta_infty =  0.14199449889115937
L2-error=velocity =  0.0010213859632319062
L2-error=pressure =  0.00032618752768789455
t= ParameterCF, val = 0.770625
beta_infty =  0.14182055140736766
L2-error=velocity =  0.0010210573285970835
L2-error=pressure =  0.0003257776232344324
t= ParameterCF, val = 0.771875
beta_infty =  0.1416463758119818
L2-error=velocity =  0.0010207272968232632
L2-error=pressure =  0.00032536727747791886
t= ParameterCF, val = 0.773125
beta_infty =  0.14147197247256044
L2-error=velocity =  0.0010203958716348061
L2-error=pressure =  0.00032495649108034707
t= ParameterCF, val = 0.774375
beta_infty =  0.141297341757063
L2-error=velocity =  0.0010200630567506687
L2-error=pressur

L2-error=velocity =  0.0010021289125970246
L2-error=pressure =  0.0003038706555047709
t= ParameterCF, val = 0.836875
beta_infty =  0.1322841170773661
L2-error=velocity =  0.0010017312533293287
L2-error=pressure =  0.0003034383280427525
t= ParameterCF, val = 0.838125
beta_infty =  0.13209838416870823
L2-error=velocity =  0.0010013323883180485
L2-error=pressure =  0.0003030055975461551
t= ParameterCF, val = 0.839375
beta_infty =  0.1319124433160969
L2-error=velocity =  0.0010009323210863382
L2-error=pressure =  0.00030257246480130777
t= ParameterCF, val = 0.840625
beta_infty =  0.13172629489467103
L2-error=velocity =  0.0010005310551552196
L2-error=pressure =  0.0003021389305969706
t= ParameterCF, val = 0.841875
beta_infty =  0.13153993927948737
L2-error=velocity =  0.0010001285940436434
L2-error=pressure =  0.0003017049957245088
t= ParameterCF, val = 0.843125
beta_infty =  0.13135337684551218
L2-error=velocity =  0.0009997249412685468
L2-error=pressure =  0.0003012706609777488
t= Parame

L2-error=velocity =  0.0009785590403174295
L2-error=pressure =  0.0002795156569635815
t= ParameterCF, val = 0.905625
beta_infty =  0.12176980500453816
L2-error=velocity =  0.000978100262087741
L2-error=pressure =  0.0002790623995514854
t= ParameterCF, val = 0.906875
beta_infty =  0.12157318830710193
L2-error=velocity =  0.0009776404703375867
L2-error=pressure =  0.000278608786359493
t= ParameterCF, val = 0.908125
beta_infty =  0.12137638404614012
L2-error=velocity =  0.00097717966856439
L2-error=pressure =  0.0002781548183292581
t= ParameterCF, val = 0.909375
beta_infty =  0.12117939258562871
L2-error=velocity =  0.000976717860266829
L2-error=pressure =  0.0002777004964056194
t= ParameterCF, val = 0.910625
beta_infty =  0.12098221428926643
L2-error=velocity =  0.0009762550489449074
L2-error=pressure =  0.00027724582153660705
t= ParameterCF, val = 0.911875
beta_infty =  0.1207848495204741
L2-error=velocity =  0.0009757912381000072
L2-error=pressure =  0.0002767907946736835
t= ParameterC

L2-error=velocity =  0.00095140382661717
L2-error=pressure =  0.00025361270234428615
t= ParameterCF, val = 0.975625
beta_infty =  0.11054736886596353
L2-error=velocity =  0.000950893678175598
L2-error=pressure =  0.0002531410676231558
t= ParameterCF, val = 0.976875
beta_infty =  0.11034475712057935
L2-error=velocity =  0.0009503827125808273
L2-error=pressure =  0.00025266913535463865
t= ParameterCF, val = 0.978125
beta_infty =  0.11014197047469079
L2-error=velocity =  0.0009498709335188715
L2-error=pressure =  0.00025219690669676053
t= ParameterCF, val = 0.979375
beta_infty =  0.10993900925267698
L2-error=velocity =  0.0009493583446797341
L2-error=pressure =  0.00025172438281204806
t= ParameterCF, val = 0.980625
beta_infty =  0.1097358737792403
L2-error=velocity =  0.0009488449497574975
L2-error=pressure =  0.0002512515648677378
t= ParameterCF, val = 0.981875
beta_infty =  0.10953256437940262
L2-error=velocity =  0.0009483307524504111
L2-error=pressure =  0.000250778454035667
t= Parame

In [21]:
Draw (u_1, mesh, interpolate_multidim=True, animate=True,
      min=-0.1, max=0.1, autoscale=False, vectors = True)
Draw (u_exact, mesh, interpolate_multidim=True, animate=True,
      min=-0.1, max=0.1, autoscale=False, vectors = True)
Draw (p1, mesh, interpolate_multidim=True, animate=True,
      min=-0.5, max=0.5, autoscale=False)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [26]:
# maxh = [0,4,0.2,0.1,0.05,0.0125]
# maxh = [0.2,0.1,0.05]
#err_v = [0.0013585343695033008,0.0003161984256015276,0.000129710622606958,2.9931526867865645e-05]
err_v = [0.0070771317262112035,0.0013829983298321504,0.0002463360046588977,7.621041160409134e-05 ]
err_p = [0.018892037636507597,0.006617070428241232,0.002237782531594837,0.0008733764520396135 ]

# L2-error=velocity =  1.385022726734725e-05
# L2-error=pressure =  0.0003538133656948336

#dt=cfl*h
maxh = [0.4, 0.2, 0.1,0.05,0.0125]

err_v = [0.007326679043085012, 0.001440017148184981, 0.00023132758590610464,5.425956529038394e-05, 3.507656384614606e-06]
err_p= [ 0.018882833182824663, 0.006617145791229282, 0.002237856769453591,0.0008732305146934806, 0.00014567024090310587]

#dt=0.0025

# maxh = [0.4, 0.2, 0.1,0.05,0.0125]

# err_v = [0.008540314415648766,0.0013856831259459496, 0.00023132758590611315, 8.806140599167779e-05, 1.7297242099983257e-05]
# err_p= [0.02430313077391392, 0.006561924242799163, 0.0022378567694535624, 0.001104248609003657,0.0002922256997266631]

#maxhs  = [0.2,0.1,0.05] 
#err_v =  [0.25180908428244286,0.06042844667099612 ]
#err_p = [ 0.05064082177944421,0.007755257351949141  ] 
#err_v =  [0.0101407566076981,0.001747502860750249,0.00044342729502770646 ]
#err_p = [0.027880013901244633,0.010022972635385803,0.006682130004079638 ]
# maxh = [0.4, 0.2, 0.1,0.05,0.025]

# err_v = [0.007326679043085012, 0.001440017148184981, 0.00023132758590610464,5.425956529038394e-05,  1.385022726734725e-05]
# err_p= [ 0.018882833182824663, 0.006617145791229282, 0.002237856769453591,0.0008732305146934806, 0.0003538133656948336]
maxh = [0.4, 0.2, 0.1,0.05,0.025]

err_v = [0.025260132627048282, 0.008016413367050368,0.002831635354724977,0.0009410498213183805]
err_p= [ 0.01209250599253369,0.004067307955726838,0.0009772437237241577,0.00024412481684383183]


In [27]:
eoc = [ log(err_v[i-1]/err_v[i])/log(2) for i in range(1,len(err_v))]
print("eoc = ", eoc)

eoc =  [1.6558334056839317, 1.5013214095476082, 1.5892924839525935]


In [24]:
eoc = [ log(err_p[i-1]/err_p[i])/log(2) for i in range(1,len(err_p))]
print("eoc = ", eoc)

eoc =  [1.5719671191948743, 2.057283909360675]


In [25]:
#  0.00023132758590610464
# 0.002237857103134506
# 0.0022378571031344634
# 0.00023132757067456503
#0.00023132757067456503
# 0.00023132757067456503
#  0.00023132757067456284

In this benchmark scenario of specific interests are the forces acting on the disk (cylinder). We will discuss how to compute them next.